In [69]:
import pycolmap 
from pathlib import Path 
import numpy as np
from scipy.spatial.transform import Rotation as R

In [68]:
scene_folder = Path('../data/south-building')
images_folder = scene_folder / 'images'

database_path = scene_folder / 'database.db'

### Sparse Reconstruction

In [21]:
pycolmap.extract_features(database_path, images_folder)
pycolmap.match_exhaustive(database_path)
maps = pycolmap.incremental_mapping(database_path, images_folder, scene_folder)
maps[0].write(scene_folder)

In [22]:
maps

{0: Reconstruction(num_reg_images=128, num_cameras=1, num_points3D=58848, num_observations=343749)}

Convert to PLY format, suitable for 3D data. (Sparse point cloud)

In [23]:
reconstruction = pycolmap.Reconstruction(scene_folder)

In [24]:
reconstruction.export_PLY(scene_folder / 'sparse.ply')

### Examining Reconstruction

In [25]:
reconstruction = pycolmap.Reconstruction(scene_folder / '0')
print(reconstruction.summary())

Reconstruction:
	num_reg_images = 128
	num_cameras = 1
	num_points3D = 58848
	num_observations = 343749
	mean_track_length = 5.8413
	mean_observations_per_image = 2685.54
	mean_reprojection_error = 0.675965


In [26]:
for image_id, image in reconstruction.images.items():
    print(image_id, image)

1 Image(image_id=1, camera_id=1, name="P1180141.JPG", triangulated=1965/8172)
2 Image(image_id=2, camera_id=1, name="P1180142.JPG", triangulated=2086/8620)
3 Image(image_id=3, camera_id=1, name="P1180143.JPG", triangulated=1955/8982)
4 Image(image_id=4, camera_id=1, name="P1180144.JPG", triangulated=1191/9548)
5 Image(image_id=5, camera_id=1, name="P1180145.JPG", triangulated=578/6590)
6 Image(image_id=6, camera_id=1, name="P1180146.JPG", triangulated=640/7354)
7 Image(image_id=7, camera_id=1, name="P1180147.JPG", triangulated=724/10824)
8 Image(image_id=8, camera_id=1, name="P1180148.JPG", triangulated=908/7044)
9 Image(image_id=9, camera_id=1, name="P1180149.JPG", triangulated=887/6434)
10 Image(image_id=10, camera_id=1, name="P1180150.JPG", triangulated=802/6822)
11 Image(image_id=11, camera_id=1, name="P1180151.JPG", triangulated=781/6926)
12 Image(image_id=12, camera_id=1, name="P1180152.JPG", triangulated=643/6952)
13 Image(image_id=13, camera_id=1, name="P1180153.JPG", triangula

In [51]:
img = reconstruction.images[1]

help(img)

Help on Image in module pycolmap object:

class Image(pybind11_builtins.pybind11_object)
 |  Method resolution order:
 |      Image
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |
 |  Methods defined here:
 |
 |  __copy__(...)
 |      __copy__(self: pycolmap.Image) -> pycolmap.Image
 |
 |  __deepcopy__(...)
 |      __deepcopy__(self: pycolmap.Image, arg0: dict) -> pycolmap.Image
 |
 |  __getstate__(...)
 |      __getstate__(self: pycolmap.Image) -> dict
 |
 |  __init__(...)
 |      __init__(*args, **kwargs)
 |      Overloaded function.
 |
 |      1. __init__(self: pycolmap.Image) -> None
 |
 |      2. __init__(self: pycolmap.Image, name: str = '', points2D: List[colmap::Point2D] = [], cam_from_world: pycolmap.Rigid3d = Rigid3d(quat_xyzw=[0, 0, 0, 1], t=[0, 0, 0]), camera_id: int = 4294967295, id: int = 4294967295) -> None
 |
 |      3. __init__(self: pycolmap.Image, name: str = '', keypoints: numpy.ndarray[numpy.float64[m, 2]] = array([], shape=(0, 2), dtype=float6

### Camera

In [40]:
help(img.cam_from_world)

Help on Rigid3d in module pycolmap object:

class Rigid3d(pybind11_builtins.pybind11_object)
 |  Method resolution order:
 |      Rigid3d
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |
 |  Methods defined here:
 |
 |  __copy__(...)
 |      __copy__(self: pycolmap.Rigid3d) -> pycolmap.Rigid3d
 |
 |  __deepcopy__(...)
 |      __deepcopy__(self: pycolmap.Rigid3d, arg0: dict) -> pycolmap.Rigid3d
 |
 |  __getstate__(...)
 |      __getstate__(self: pycolmap.Rigid3d) -> dict
 |
 |  __init__(...)
 |      __init__(*args, **kwargs)
 |      Overloaded function.
 |
 |      1. __init__(self: pycolmap.Rigid3d) -> None
 |
 |      2. __init__(self: pycolmap.Rigid3d, arg0: pycolmap.Rotation3d, arg1: numpy.ndarray[numpy.float64[3, 1]]) -> None
 |
 |      3. __init__(self: pycolmap.Rigid3d, arg0: numpy.ndarray[numpy.float64[3, 4]]) -> None
 |
 |      4. __init__(self: pycolmap.Rigid3d, arg0: dict) -> None
 |
 |      5. __init__(self: pycolmap.Rigid3d, **kwargs) -> None
 |
 |  __mul_

#### Extrinsic Matrix 
These are parameters that do not depend on the camera, they include **rotation** and **translation** (which don't depend on camera's build). The rotation can be expressed by quarternion (that uses only 4x1 matrix instead of 3x3 matrix). The translation vector tells the camera's position relative to the origin. 

It is used to translate world coordinate system to camera coordinate system. 

In [49]:
print(img.cam_from_world)

Rigid3d(quat_xyzw=[0.0179487, 0.405851, -0.120835, 0.905738], t=[-0.668025, 1.00101, 3.64116])


In [57]:
print(reconstruction.images[10].cam_from_world)

Rigid3d(quat_xyzw=[0.0107518, 0.619459, -0.177943, 0.76452], t=[-0.500664, 0.985502, 3.52476])


In [42]:
print(img.cam_from_world.matrix())

[[ 0.64136752  0.23345888  0.73085201 -0.66802545]
 [-0.20432089  0.97015346 -0.13059573  1.00101326]
 [-0.73952733 -0.06556847  0.66992544  3.64116403]]


Essential matrix encapsulates the relative rotation and translation between two camera views.

In [39]:
print(img.cam_from_world.essential_matrix())

[[ 0.00096201 -0.93826036  0.2988679 ]
 [ 0.48014481  0.21024363  0.81063118]
 [-0.13182287 -0.22993703 -0.16802352]]


In [48]:
img.cam_from_world.rotation.quat # rotation quaternion

array([ 0.01794869,  0.40585111, -0.12083508,  0.90573816])

In [44]:
img.cam_from_world.translation # translation vector

array([-0.66802545,  1.00101326,  3.64116403])

In [58]:
reconstruction.images[10].cam_from_world.translation

array([-0.5006644 ,  0.98550227,  3.52475678])

In [50]:
for camera_id, camera in reconstruction.cameras.items():
    print(camera_id, camera)

1 Camera(camera_id=1, model=SIMPLE_RADIAL, width=3072, height=2304, params=[2558.022368, 1536.000000, 1152.000000, -0.020224] (f, cx, cy, k))


#### Intrinsic matrix
This is needed to perform transformation between a point P in the 3D camera reference system to a point P' in the 2D image plane.

 \begin{pmatrix}
 fk & 0 & c_x & 0 \\
 0 & fk & c_y & 0 \\
 0 & 0 & 1 & 0 
 \end{pmatrix}
 
- f : focal length
- k : ratio $\frac{pixels}{cm}$ (digital images are expressed in pixels)
- $c_x$, $c_y$ : translation vector that describes offset between digital and camera coordinates

(this form of matrix is needed to make translations on 3D points)

In [29]:
reconstruction.cameras

{1: Camera(camera_id=1, model=SIMPLE_RADIAL, width=3072, height=2304, params=[2558.022368, 1536.000000, 1152.000000, -0.020224] (f, cx, cy, k))}

In [27]:
for point3D_id, point3D in reconstruction.points3D.items():
    print(point3D_id, point3D)

944 Point3D(xyz=[-0.376446, -0.354972, -0.423369], color=[179, 148, 126], error=0.0710824, track=Track(length=2))
60830 Point3D(xyz=[1.16903, -0.303112, -0.389866], color=[201, 168, 151], error=0.758434, track=Track(length=11))
1 Point3D(xyz=[0.226638, -1.98234, 0.413007], color=[118, 110, 126], error=0.113303, track=Track(length=3))
2 Point3D(xyz=[0.269489, -1.95152, 0.39632], color=[91, 79, 99], error=0.341881, track=Track(length=4))
40859 Point3D(xyz=[-2.06854, 0.206151, 1.79784], color=[175, 147, 127], error=0.281243, track=Track(length=5))
3517 Point3D(xyz=[-0.675873, -0.215625, -0.393756], color=[169, 144, 136], error=0.598742, track=Track(length=8))
393 Point3D(xyz=[-0.256086, -0.78631, -0.322062], color=[208, 183, 176], error=0.414255, track=Track(length=3))
37879 Point3D(xyz=[-1.32412, 0.0541832, 1.67734], color=[168, 150, 138], error=0.506565, track=Track(length=13))
1054 Point3D(xyz=[0.259905, -1.34173, -0.109003], color=[114, 103, 101], error=0.420951, track=Track(length=14

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



### Translation to image plane

In [59]:
point = reconstruction.points3D[1]

In [60]:
help(point)

Help on Point3D in module pycolmap object:

class Point3D(pybind11_builtins.pybind11_object)
 |  Method resolution order:
 |      Point3D
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |
 |  Methods defined here:
 |
 |  __copy__(...)
 |      __copy__(self: pycolmap.Point3D) -> pycolmap.Point3D
 |
 |  __deepcopy__(...)
 |      __deepcopy__(self: pycolmap.Point3D, arg0: dict) -> pycolmap.Point3D
 |
 |  __getstate__(...)
 |      __getstate__(self: pycolmap.Point3D) -> dict
 |
 |  __init__(...)
 |      __init__(*args, **kwargs)
 |      Overloaded function.
 |
 |      1. __init__(self: pycolmap.Point3D) -> None
 |
 |      2. __init__(self: pycolmap.Point3D, arg0: dict) -> None
 |
 |      3. __init__(self: pycolmap.Point3D, **kwargs) -> None
 |
 |  __repr__(...)
 |      __repr__(self: pycolmap.Point3D) -> str
 |
 |  __setstate__(...)
 |      __setstate__(self: pycolmap.Point3D, arg0: dict) -> None
 |
 |  mergedict(...)
 |      mergedict(self: object, arg0: dict) -> None
 

In [64]:
points = np.array([point.xyz for point in reconstruction.points3D.values()]) 
points

array([[-0.37644627, -0.35497232, -0.42336873],
       [ 1.1690313 , -0.3031122 , -0.38986629],
       [ 0.2266383 , -1.98233931,  0.41300728],
       ...,
       [ 1.77361289,  0.35646101,  0.80923833],
       [ 1.66443585,  0.13557404,  0.19738035],
       [ 1.67394656,  0.19428105,  0.22961195]])

In [66]:
img = reconstruction.images[1]
print(img.cam_from_world.rotation.quat)
print(img.cam_from_world.translation)

[ 0.01794869  0.40585111 -0.12083508  0.90573816]
[-0.66802545  1.00101326  3.64116403]


In [70]:
rot = R.from_quat(img.cam_from_world.rotation.quat)

In [72]:
rot.as_matrix()

array([[ 0.64136752,  0.23345888,  0.73085201],
       [-0.20432089,  0.97015346, -0.13059573],
       [-0.73952733, -0.06556847,  0.66992544]])

In [93]:
rotation_part = np.concatenate((rot.as_matrix(), [[0, 0, 0]]))
translation_part = np.concatenate((img.cam_from_world.translation, [1])).reshape(4, 1)

In [94]:
print(rotation_part)
print(translation_part)
print(rotation_part.shape)
print(translation_part.shape)

[[ 0.64136752  0.23345888  0.73085201]
 [-0.20432089  0.97015346 -0.13059573]
 [-0.73952733 -0.06556847  0.66992544]
 [ 0.          0.          0.        ]]
[[-0.66802545]
 [ 1.00101326]
 [ 3.64116403]
 [ 1.        ]]
(4, 3)
(4, 1)


In [95]:
extrinsic_matrix = np.concatenate((rotation_part, translation_part), axis=1)
extrinsic_matrix

array([[ 0.64136752,  0.23345888,  0.73085201, -0.66802545],
       [-0.20432089,  0.97015346, -0.13059573,  1.00101326],
       [-0.73952733, -0.06556847,  0.66992544,  3.64116403],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [100]:
ps = np.concatenate((points, np.ones((len(points), 1), np.float32)), axis=1)
ps

array([[-0.37644627, -0.35497232, -0.42336873,  1.        ],
       [ 1.1690313 , -0.3031122 , -0.38986629,  1.        ],
       [ 0.2266383 , -1.98233931,  0.41300728,  1.        ],
       ...,
       [ 1.77361289,  0.35646101,  0.80923833,  1.        ],
       [ 1.66443585,  0.13557404,  0.19738035,  1.        ],
       [ 1.67394656,  0.19428105,  0.22961195,  1.        ]])

In [101]:
ps @ extrinsic_matrix

array([[ 0.1441806 , -0.40450271, -0.51239412, -0.64541131],
       [ 1.10002764,  0.00441832,  0.63279268, -1.50392911],
       [ 0.2449616 , -1.89734287,  0.7012082 ,  0.36807919],
       ...,
       [ 0.46625142,  0.70682704,  1.7918256 ,  3.11857314],
       [ 0.89384633,  0.50716302,  1.330981  ,  0.74252013],
       [ 0.86411497,  0.56422481,  1.35185781,  0.91229376]])

In [110]:
print(reconstruction.cameras[1])

Camera(camera_id=1, model=SIMPLE_RADIAL, width=3072, height=2304, params=[2558.022368, 1536.000000, 1152.000000, -0.020224] (f, cx, cy, k))


In [112]:
f, cx, cy, k = reconstruction.cameras[1].params
intrinsic_matrix = np.array([[f * k, 0, cx, 0], [0, f*k, cy, 0], [0, 0, 1, 0]])

In [113]:
intrinsic_matrix

array([[-5.1732729e+01,  0.0000000e+00,  1.5360000e+03,  0.0000000e+00],
       [ 0.0000000e+00, -5.1732729e+01,  1.1520000e+03,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  1.0000000e+00,  0.0000000e+00]])

In [114]:
projected_to_camera_viewpoint = (ps @ extrinsic_matrix).T

In [118]:
projected_to_image_plane = (intrinsic_matrix @ projected_to_camera_viewpoint).T

In [117]:
projected_to_image_plane

array([[-7.94496227e+02,  9.15062130e+02,  1.06438327e+03, ...,
         2.72812366e+03,  1.99814571e+03,  2.03175057e+03],
       [-5.69351999e+02,  7.28748599e+02,  9.05946572e+02, ...,
         2.02761700e+03,  1.50705319e+03,  1.52815131e+03],
       [-5.12394122e-01,  6.32792683e-01,  7.01208201e-01, ...,
         1.79182560e+00,  1.33098100e+00,  1.35185781e+00]])